# Conections invariant under the involution

Let $C$ be the elliptic curve is given by $y^2 = P(x)$, where $P(x) = x(x-1)(x-\lambda)$. We consider here connections on the trivial bundle $\mathcal{O}_C\oplus\mathcal{O}_C$ having two logarithmic singularities over two points in involution $t_1,t_2$ and two apparent singularities over another pair of points in involution $p_1,p_2$.  

In [1]:
import re
from IPython.core.display import display, HTML, Math

# Define the width of the cells
display(HTML("<style>.container { width:85% !important; }</style>"))

def DM(expression):
    """Short for DisplayMath: converts symbolic expression into latex and displays it nicely"""
    return display(Math(latex(expression)))

In [3]:
# We use the following variables:
var('x y z L t r p q u n a_0 a_1 a_2 b_0 b_1 b_2 b_3 c0 c1 c2')
variables = [x, y, L, t, r, p, q, u, n]
parameters = [a_0, a_1, a_2, b_0, b_1, b_2, b_3]

In [4]:
# Some concrete values that can be used for testing
test_vals = [L == 1219/84, p == 3/7, q == 13/7, n == 1/7, t == 2/3, u == 5]
rval = [r==sqrt((x*(x-1)*(x-L)).subs(x==t))]
test_vals.append(r == r.subs(rval).subs(test_vals))
print test_vals

[L == (1219/84), p == (3/7), q == (13/7), n == (1/7), t == (2/3), u == 5, r == 1/3*sqrt(1163/42)]


| Variable | Meaning |
| :---: | --- |
| `(x,y)` | Coordinates on $\mathbb{C}^2$ |
| `P` | The polynomial $x(x-1)(x-\lambda)$ defining the curve $C$ |
| `z` | Coordinate on the (projectivized) fiber is $[1:z]$ |
| `L` | The number $\lambda$ defining the curve |
| `(t,r)` | The coordinates of the point $t_1$ |
| `(p,q)` | The coordinates of the point $p_1$ |
| `u` | $z$-coordinate of the parabolic point over $t_1$ |
| `n` | Stands for $\eta$, the exponent of the singularity over $t_1$ |
| `a_i, b_i` | Parameters defining a connection |
| `c0, c1, c2` | Coefficients used to combine $\nabla_0$, $\Theta_1$ and $\Theta_2$ |

A connection in our family is given as

$$\nabla = d + \frac{M}{(x-p)(x-t)}\frac{dx}{y}, \quad M=\pmatrix{
    \alpha & \beta \\
    \gamma & -\alpha \\
},$$

where $\alpha = a_0 + a_1x + a_2x^2 + a_3y$, and so on.

### Invariance conditions 

We choose to use the following normal form:  
`a3=0, c3=b3, c2=-b2, c1=-b1, c0=-b0.`  

This means that the connection $\nabla$ is invariant by the following transformation: Pull back $\nabla$ by the elliptic involution and exchange the order of the standard basis of $\mathcal{O}_C\oplus\mathcal{O}_C$. In particular the parabolic over $t_2$ is located at `1/u`.

In [5]:
a_3=0
c_3=b_3
c_2=-b_2
c_1=-b_1
c_0=-b_0

In [6]:
a = a_0 + a_1*x + a_2*x^2 + a_3*y
b = b_0 + b_1*x + b_2*x^2 + b_3*y
c = c_0 + c_1*x + c_2*x^2 + c_3*y

In [7]:
M = matrix([[a, b], [c, -a]])
P = x*(x-1)*(x-L)
dP = diff(P, x)

In [8]:
# Function to simplify powers of q and r (up to degree 4)

q2 = P.subs(x==p)
r2 = P.subs(x==t)

# Create a dictionary to pass onto the 'sage_eval' command
dict = {'r2': r2, 'q2':q2}
for v in variables + parameters:
    dict[str(v)] = v

def simplify_qr(expr):
    """Convert expr to a string and replace powers of q and r"""
    global dict
    expr = expr.simplify_rational()
    num = expr.numerator().expand()
    str_num = str(num)
    str_num = re.sub('q\^2', 'q2', str_num)
    str_num = re.sub('q\^3', 'q*q2', str_num)
    str_num = re.sub('q\^4', 'q2^2', str_num)
    str_num = re.sub('r\^2', 'r2', str_num)
    str_num = re.sub('r\^3', 'r*r2', str_num)
    str_num = re.sub('r\^4', 'r2^2', str_num)
    
    den = expr.denominator().expand()
    str_den = str(den)
    str_den = re.sub('q\^2', 'q2', str_den)
    str_den = re.sub('q\^3', 'q*q2', str_den)
    str_den = re.sub('q\^4', 'q2^2', str_den)
    str_den = re.sub('r\^2', 'r2', str_den)
    str_den = re.sub('r\^3', 'r*r2', str_den)
    str_den = re.sub('r\^4', 'r2^2', str_den)
    
    new_expr = sage_eval(str_num, locals=dict)/sage_eval(str_den, locals=dict)
    new_expr = new_expr.simplify_rational()
    if new_expr == 0:
        return new_expr
    else:
        return new_expr.factor()

## Computation of the residues

The residues are given by the following formulas:

$\operatorname{Res}_{p_i}\nabla = \operatorname{Res}_{p_i} \frac{M}{y(x-t)} \frac{dx}{x-p} = \frac{M(p_i)}{\pm q(p-t)}$

$\operatorname{Res}_{t_i}\nabla = \operatorname{Res}_{t_i} \frac{M}{y(x-p)} \frac{dx}{x-t} = \frac{M(t_i)}{\pm r(t-p)}$

In [9]:
p1 = [x==p, y==q]
p2 = [x==p, y==-q]
t1 = [x==t, y==r]
t2 = [x==t, y==-r]
Sing = [p1, p2, t1, t2]

In [10]:
Res = ['']*4

Res[0] = (M/(y*(x-t))).subs(p1)
Res[1] = (M/(y*(x-t))).subs(p2)
Res[2] = (M/(y*(x-p))).subs(t1)
Res[3] = (M/(y*(x-p))).subs(t2)

## Linear conditions on the type of singularities

The parameters `a_i, b_i` are subject to the following constrains:

* The residue at $p_1$ has `(0,1)` as a parabolic direction with eigenvalue `1/2`,
* The residue at $t_1$ has `(1,u)` as parabolic direction with eigenvalue `n/2`.

By symmetry, this also fixes the type of singularities over $p_2$ and $t_2$.

In [11]:
# We get 5 equations besides the invariance conditions
Eq = [0]*5

Parabolic direction and eigenvalue over $p_1$:

In [12]:
Eq[0] = Res[0][0][1].numerator()
Eq[1] = (Res[0][1][1] - 1/2).simplify_rational().numerator()

Parabolic direction and eigenvalue over $t_1$:

In [13]:
V1 = Res[2]*vector([1, u]) - (n/2)*vector([1, u])

In [14]:
Eq[2] = V1[0].simplify_rational().numerator()
Eq[3] = V1[1].simplify_rational().numerator()

There is one more linear condition that guarantees that the singularity over $p_1$ is apparent. To find it, let us expand $y=\sqrt{P(x)}$ as a power series near $p_1$:  
$ y(x) = q + \frac{1}{2q}P'(p)(x-p) + \ldots$

In [15]:
# reacall: P = x*(x-1)*(x-L)
l = diff(P, x).subs(x==p).expand()
y_truncated = q + l*(x-p)/(2*q)

In [16]:
C01 = (M[0][1]/(y*(x-t))).subs(y==y_truncated)

In [17]:
constant_term = diff(C01, x).subs(x==p)

In [18]:
# Simplify q^2 and q^3 using 'simplify_qr'
Eq[4] = simplify_qr(constant_term).numerator() 

### The five linear conditions:

In [19]:
for e in Eq:
    print e.factor(), '\n'

b_2*p^2 + b_1*p + b_3*q + b_0 

-2*a_2*p^2 - 2*a_1*p - p*q + q*t - 2*a_0 

-2*b_2*t^2*u - n*p*r + n*r*t - 2*a_2*t^2 - 2*b_3*r*u - 2*b_1*t*u - 2*a_1*t - 2*b_0*u - 2*a_0 

-n*p*r*u + n*r*t*u + 2*a_2*t^2*u + 2*b_2*t^2 + 2*a_1*t*u - 2*b_3*r + 2*b_1*t + 2*a_0*u + 2*b_0 

b_2*p^5 - 2*L*b_2*p^3*t + b_2*p^4*t - 2*L*b_1*p^3 - L*b_2*p^3 + 3*b_1*p^4 - 2*L*b_3*p^2*q + 2*b_3*p^3*q + 3*L*b_2*p^2*t - b_1*p^3*t - 2*b_2*p^3*t - 4*L*b_0*p^2 + L*b_1*p^2 + 5*b_0*p^3 - 2*b_1*p^3 + 2*L*b_3*p*q - 2*b_3*p^2*q + 2*L*b_0*p*t + L*b_1*p*t - 3*b_0*p^2*t + 3*L*b_0*p - 4*b_0*p^2 - L*b_0*t + 2*b_0*p*t 



In [20]:
Sols = solve(Eq, [a_0, a_1, b_0, b_1, b_2])[0]  # a_2, b_3 are free

In [21]:
# The values of the parameters in terms of a_2, b_3
for v in parameters:
    if v.subs(Sols) == 0:
        print '{} = 0'.format(v), '\n'
    else:
        print '{} ='.format(v), v.subs(Sols).simplify_rational().factor(), '\n'

a_0 = 1/2*(n*p^2*r*u^2 + 2*a_2*p^2*t*u^2 - n*p*r*t*u^2 - 2*a_2*p*t^2*u^2 + p*q*t*u^2 - q*t^2*u^2 + n*p^2*r - 2*a_2*p^2*t - n*p*r*t + 2*a_2*p*t^2 + 4*b_3*p*r*u - p*q*t + q*t^2)/((p - t)*(u + 1)*(u - 1)) 

a_1 = -1/2*(2*a_2*p^2*u^2 + n*p*r*u^2 - n*r*t*u^2 - 2*a_2*t^2*u^2 + p*q*u^2 - q*t*u^2 - 2*a_2*p^2 + n*p*r - n*r*t + 2*a_2*t^2 + 4*b_3*r*u - p*q + q*t)/((p - t)*(u + 1)*(u - 1)) 

a_2 = a_2 

b_0 = -1/2*(2*L*n*p^4*r*u - 2*n*p^5*r*u - 2*L*n*p^3*r*t*u + 2*n*p^4*r*t*u + 2*L*b_3*p^3*r*u^2 - 2*b_3*p^4*r*u^2 - 2*L*b_3*p^2*q*t*u^2 + b_3*p^3*q*t*u^2 + b_3*p^2*q*t^2*u^2 - 2*L*n*p^3*r*u + 2*n*p^4*r*u + 2*L*n*p^2*r*t*u - 2*n*p^3*r*t*u - 2*L*b_3*p^2*r*u^2 + 2*b_3*p^3*r*u^2 + 3*L*b_3*p*q*t*u^2 - 2*b_3*p^2*q*t*u^2 - L*b_3*q*t^2*u^2 + 2*L*b_3*p^3*r - 2*b_3*p^4*r + 2*L*b_3*p^2*q*t - b_3*p^3*q*t - b_3*p^2*q*t^2 - 2*L*b_3*p^2*r + 2*b_3*p^3*r - 3*L*b_3*p*q*t + 2*b_3*p^2*q*t + L*b_3*q*t^2)/((L - p)*(p - t)^2*(p - 1)*(u + 1)*(u - 1)) 

b_1 = 1/2*(4*L*n*p^4*r*u - 4*n*p^5*r*u - 4*L*n*p^3*r*t*u + 4*n*p^4*r*t*u

**Remark:** The parameter `a_2` does not appear in the residues !

In [22]:
for k in range(4):
    print 'Residue at {}:'.format(Sing[k])
    DM(Res[k].subs(Sols).simplify_rational().factor())
    print ''

Residue at [x == p, y == q]:


<IPython.core.display.Math object>


Residue at [x == p, y == -q]:


<IPython.core.display.Math object>


Residue at [x == t, y == r]:


<IPython.core.display.Math object>


Residue at [x == t, y == -r]:


<IPython.core.display.Math object>

In [23]:
# Position of the eight singularities:
for k in range(4):
    print Res[k].subs(Sols).simplify_rational().factor().eigenvectors_right()

[(-1/2, [(1, -2*b_3/(p - t))], 1), (1/2, [(0, 1)], 1)]
[(-1/2, [(1, -1/2*(p - t)/b_3)], 1), (1/2, [(1, 0)], 1)]
[(-1/2*n, [(1, (n*p - n*t + 2*b_3*u)/((n*p - n*t)*u + 2*b_3))], 1), (1/2*n, [(1, u)], 1)]
[(-1/2*n, [(1, ((n*p - n*t)*u + 2*b_3)/(n*p - n*t + 2*b_3*u))], 1), (1/2*n, [(1, 1/u)], 1)]


In [24]:
# Values of b_3 needed to get singularities at z = +-1

print solve((n*p - n*t + 2*b_3*u)/((n*p - n*t)*u + 2*b_3) - 1, b_3)[0]
print solve((n*p - n*t + 2*b_3*u)/((n*p - n*t)*u + 2*b_3) + 1, b_3)[0]

b_3 == 1/2*n*p - 1/2*n*t
b_3 == -1/2*n*p + 1/2*n*t


# Looking for the right decomposition $\nabla = \nabla_0^+ + c_1\Theta_1^+ + c_2\Theta_2^+$

We aim to define $\nabla_0^+$, $\Theta_1^+$, $\Theta_2^+$ which are characterized by a geometric property. Throught this section we will work exclusively with the apparent map with respect to the subbundle $z=1$, and  we keep the superscript "+" to highlight this fact.

We define objects characterized by the following properties:

* $\nabla_0^+$ is the only connection with apparent singularities at $x=\lambda$ and $x=t$.
* $\Theta_1^+$ is the only Higgs field such that the connection $\nabla_0^+ - \Theta_1$ has apparent singularities at $w_\infty$ and $x=\lambda$.
* $\Theta_2^+$ is the only Higgs field such that the connection $\nabla_0^+ - \Theta_2$ has apparent singularities at $w_\infty$ and $x=t$.

**Remark:** This definition of the $\Theta_i^+$ differs from the one in Loray-Saito by a constant scalar factor. The advantage of this rescaling is that now the apparent map is given by:

$$ \operatorname{App}(\nabla) = (x-\lambda)(x-t) + c_1(x-\lambda)(x-\mu_1) + c_2(x-t)(x-\mu_2), $$

where $\mu_i$ is the (second) apparent singularity of $\Theta_i^+$, and is computed in Loray-Saito as $\mu_i = t_i(u_i-1)/(u_i-t_i)$. 

We will later compute the values of $\mu_i$ in terms of our variables, but right now we can already point out that $\mu_2=p$.

## Apparent map with respect to the section $\{z=1\}$

We define an apparent map giving the divisor of tangencies between $\nabla$ and the subbundle $z=1$. The divisor is of degree 4 and always contains two pairs of points in involution.

In [25]:
# Recall that M is given as follos:
DM(M)

<IPython.core.display.Math object>

Therefore we have that

$\operatorname{App}(x,y) = 2(a_2 + b_2)x^2 + 2(a_1 + b_1)x + 2(a_0 + b_0).$

Now we compute an explicit formula:

In [26]:
vect = (M.subs(Sols)).simplify_rational()*vector((1,1))
APPplus = (vect[0] - vect[1]).simplify_rational().numerator().factor()

### Correct values of $b_3$ and $a_2$ needed to define $\nabla_0^+$

We seek for the conection $\nabla_0^+$ which is characterized as the only connection with $\operatorname{App}(\nabla_0^+) \sim (x-t)(x-\lambda)$. In particular, this implies that the second singularities on the fibers over $t_i$ are on $z=1$, and the Riccati foliation has a double tangency with the line $\{z=1\}$ at $w_\lambda$.

In [27]:
# Value of b_3 needed to have the second singularity over t1 at z=1 is:
B3plus = 1/2*(p - t)*n

In [28]:
# Verify that APP vanishes at x=t when the value of b_3 is given by B3plus defined above
APPplus.subs(b_3 == B3plus).factor().subs(x==t)

0

We now find the correct value of $a_2$ such that APP vanishes at `x = L`

In [29]:
A2plus = a_2.subs(solve(APPplus.subs([x == L, b_3 == B3plus]), a_2))
A2plus = A2plus.simplify_rational().factor()
A2plus

1/4*(n*p^3*q*u + 2*L*n*p^2*r*u - 2*n*p^3*r*u - 2*L*n*p*q*t*u + n*p^2*q*t*u - n*p^3*q + 2*L*n*p^2*r - 2*n*p^3*r + 2*L*n*p*q*t - n*p^2*q*t + L*n*p*q*u - 2*n*p^2*q*u + 2*p^3*q*u - 2*L*n*p*r*u + 2*n*p^2*r*u + L*n*q*t*u - 2*p^2*q*t*u - L*n*p*q + 2*n*p^2*q - 2*p^3*q - 2*L*n*p*r + 2*n*p^2*r - L*n*q*t + 2*p^2*q*t - 2*p^2*q*u + 2*p*q*t*u + 2*p^2*q - 2*p*q*t)/((L - p)*(p - t)*(p - 1)*p*(u - 1))

In [30]:
# Verify that these choices of a_2, b_3 are the right ones:
APPplus.subs([a_2 == A2plus, b_3 == B3plus]).factor()

(L - x)*(n + 1)*(p - t)^2*(p - 1)*p*q*(t - x)*(u + 1)*(u - 1)

## Analysis of $\nabla_0^+$

In [31]:
nabla0plus = solve(Eq + [b_3 - B3plus, a_2 - A2plus], parameters)[0]  # This is a list containing the correct value of the parameters a_i, b_i
Nabla0plus = M.subs(nabla0plus).simplify_rational().factor()  # This is the matrix M with those correct values

In [32]:
# The values of the parameters defining Nabla0plus
for v in parameters:
    if v.subs(nabla0plus) == 0:
        print '{} = 0'.format(v), '\n'
    else:
        print '{} ='.format(v), v.subs(nabla0plus).simplify_rational().factor(), '\n'

a_0 = 1/4*(2*L*n*p^3*r*u - 2*n*p^4*r*u + n*p^3*q*t*u - 2*L*n*p*q*t^2*u + n*p^2*q*t^2*u + 2*L*n*p^3*r - 2*n*p^4*r - n*p^3*q*t + 2*L*n*p*q*t^2 - n*p^2*q*t^2 - 2*L*n*p^2*r*u + 2*n*p^3*r*u + L*n*p*q*t*u + 2*L*p^2*q*t*u - 2*n*p^2*q*t*u + L*n*q*t^2*u - 2*L*p*q*t^2*u - 2*L*n*p^2*r + 2*n*p^3*r - L*n*p*q*t - 2*L*p^2*q*t + 2*n*p^2*q*t - L*n*q*t^2 + 2*L*p*q*t^2 - 2*L*p*q*t*u + 2*L*q*t^2*u + 2*L*p*q*t - 2*L*q*t^2)/((L - p)*(p - t)*(p - 1)*(u - 1)) 

a_1 = -1/4*(n*p^4*q*u + 4*L*n*p^3*r*u - 4*n*p^4*r*u - 2*L*n*p^2*q*t*u + 2*n*p^3*q*t*u - 2*L*n*p*q*t^2*u + n*p^2*q*t^2*u - n*p^4*q + 4*L*n*p^3*r - 4*n*p^4*r + 2*L*n*p^2*q*t - 2*n*p^3*q*t + 2*L*n*p*q*t^2 - n*p^2*q*t^2 + L*n*p^2*q*u + 2*L*p^3*q*u - 2*n*p^3*q*u - 4*L*n*p^2*r*u + 4*n*p^3*r*u + 2*L*n*p*q*t*u - 2*L*p^2*q*t*u - 2*n*p^2*q*t*u + 2*p^3*q*t*u + L*n*q*t^2*u - 2*p^2*q*t^2*u - L*n*p^2*q - 2*L*p^3*q + 2*n*p^3*q - 4*L*n*p^2*r + 4*n*p^3*r - 2*L*n*p*q*t + 2*L*p^2*q*t + 2*n*p^2*q*t - 2*p^3*q*t - L*n*q*t^2 + 2*p^2*q*t^2 - 2*L*p^2*q*u + 2*L*p*q*t*u - 2*p^2*

In [33]:
# Residues of Nabla0plus
for k in range(4):
    print 'Residue at {}:'.format(Sing[k])
    residue = Res[k].subs(nabla0plus).simplify_rational().factor()
    DM(residue)
    print 'Eigenvectors:', residue.eigenvectors_right()
    print '\n'

Residue at [x == p, y == q]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2, [(1, -n)], 1), (1/2, [(0, 1)], 1)]


Residue at [x == p, y == -q]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2, [(1, -1/n)], 1), (1/2, [(1, 0)], 1)]


Residue at [x == t, y == r]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2*n, [(1, 1)], 1), (1/2*n, [(1, u)], 1)]


Residue at [x == t, y == -r]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2*n, [(1, 1)], 1), (1/2*n, [(1, 1/u)], 1)]




## Apparent map once more

I will first define naive candidates for $\Theta_1$ and $\Theta_2$ and then, via the apparent map, figure out which linear combinations of these are the right ones. 

In [34]:
to_zero = [a_0==0, a_1==0, a_2==0, b_0==0, b_1==0, b_2==0, b_3==0]
EqHom = [eq - eq.subs(to_zero) for eq in Eq]

In [35]:
theta1_naive = solve(EqHom + [b_3 - 1, a_2], parameters)[0]  # set b3=1, a2=0
theta2_naive = solve(EqHom + [b_3, a_2 - 1], parameters)[0]  # set b3=0, a2=1

In [36]:
Theta1_n = M.subs(theta1_naive).simplify_rational().factor()
Theta2_n = M.subs(theta2_naive).simplify_rational().factor()

In [37]:
Mbis = (c0*Nabla0plus + c1*Theta1_n + c2*Theta2_n).simplify_rational().factor()

In [38]:
vect = Mbis*vector((1,1))
APP_naive = (vect[0] - vect[1]).simplify_rational().numerator().factor()

In [39]:
# Check: this should return c1=0, c2=0
solve([APP_naive.subs(x==t), APP_naive.subs(x==L)], [c1, c2])

[[c1 == 0, c2 == 0]]

In [40]:
# Theta2 is defined by having apparent singularities at infinity and t
SolsTheta2 = solve([diff(diff(APP_naive, x), x), APP_naive.subs(x==t), c0 - 1], [c0, c1, c2])[0]
print SolsTheta2

[c0 == 1, c1 == 0, c2 == -1/2*(n + 1)*q/(L - p)]


In [41]:
# Theta1 is defined by having apparent singularities at infinity and L
SolsTheta1 = solve([diff(diff(APP_naive, x), x), APP_naive.subs(x==L), c0 - 1], [c0, c1, c2])[0]

print 'c0 =', c0.subs(SolsTheta1), '\n'
print 'c1 =', c1.subs(SolsTheta1).simplify_rational().factor(), '\n'
print 'c2 =', c2.subs(SolsTheta1).simplify_rational().factor(), '\n'

c0 = 1 

c1 = -1/2*(L - t)*(n + 1)*(p - t)*q*(u + 1)/(L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t) 

c2 = -1/4*(L*p^3*q*u^2 + 2*L^2*p^2*r*u^2 - 4*L*p^3*r*u^2 + 2*p^4*r*u^2 - 2*L^2*p*q*t*u^2 + L*p^2*q*t*u^2 - p^3*q*t*u^2 + 2*L*p*q*t^2*u^2 - p^2*q*t^2*u^2 + 4*L*p^3*r*u - 4*p^4*r*u - 4*L*p^2*r*t*u + 4*p^3*r*t*u + L^2*p*q*u^2 - 2*L*p^2*q*u^2 - 2*L^2*p*r*u^2 + 4*L*p^2*r*u^2 - 2*p^3*r*u^2 + L^2*q*t*u^2 - L*p*q*t*u^2 + 2*p^2*q*t*u^2 - L*q*t^2*u^2 - L*p^3*q + 2*L^2*p^2*r - 4*L*p^3*r + 2*p^4*r + 2*L^2*p*q*t - L*p^2*q*t + p^3*q*t - 2*L*p*q*t^2 + p^2*q*t^2 - 4*L*p^2*r*u + 4*p^3*r*u + 4*L*p*r*t*u - 4*p^2*r*t*u - L^2*p*q + 2*L*p^2*q - 2*L^2*p*r + 4*L*p^2*r - 2*p^3*r - L^2*q*t + L*p*q*t - 2*p^2*q*t + L*q*t^2)*(n + 1)*q/((L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)*(L - p)*(p - t)*(p - 1)*p*(u - 1)) 



# Definition of $\Theta_1^+$ and $\Theta_2^+$

In [42]:
theta1plus = solve(EqHom + [b_3 + c1.subs(SolsTheta1), a_2 + c2.subs(SolsTheta1)], parameters)[0]
Theta1plus = M.subs(theta1plus).simplify_rational().factor()

In [43]:
theta2plus = solve(EqHom + [b_3, a_2 + c2.subs(SolsTheta2)], parameters)[0]
Theta2plus = M.subs(theta2plus).simplify_rational().factor()

### Coefficients defining $\Theta_1^+$

In [44]:
for v in parameters:
    if v.subs(theta1plus) == 0:
        print '{} = 0'.format(v), '\n'
    else:
        print '{} ='.format(v), v.subs(theta1plus).simplify_rational().factor(), '\n'

a_0 = 1/4*(L*p^3*q*t*u^2 + 2*L^2*p^2*r*t*u^2 - 4*L*p^3*r*t*u^2 + 2*p^4*r*t*u^2 - 2*L^2*p*q*t^2*u^2 + L*p^2*q*t^2*u^2 - p^3*q*t^2*u^2 + 2*L*p*q*t^3*u^2 - p^2*q*t^3*u^2 + 4*L^2*p^3*r*u - 4*L*p^4*r*u - 4*L^2*p^2*r*t*u + 4*L*p^3*r*t*u + L^2*p*q*t*u^2 - 2*L*p^2*q*t*u^2 - 2*L^2*p*r*t*u^2 + 4*L*p^2*r*t*u^2 - 2*p^3*r*t*u^2 + L^2*q*t^2*u^2 - L*p*q*t^2*u^2 + 2*p^2*q*t^2*u^2 - L*q*t^3*u^2 - L*p^3*q*t + 2*L^2*p^2*r*t - 4*L*p^3*r*t + 2*p^4*r*t + 2*L^2*p*q*t^2 - L*p^2*q*t^2 + p^3*q*t^2 - 2*L*p*q*t^3 + p^2*q*t^3 - 4*L^2*p^2*r*u + 4*L*p^3*r*u + 4*L^2*p*r*t*u - 4*L*p^2*r*t*u - L^2*p*q*t + 2*L*p^2*q*t - 2*L^2*p*r*t + 4*L*p^2*r*t - 2*p^3*r*t - L^2*q*t^2 + L*p*q*t^2 - 2*p^2*q*t^2 + L*q*t^3)*(n + 1)*q/((L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)*(L - p)*(p - t)*(p - 1)*(u - 1)) 

a_1 = -1/4*(L*p^4*q*u^2 + 2*L^2*p^3*r*u^2 - 4*L*p^4*r*u^2 + 2*p^5*r*u^2 - 2*L^2*p^2*q*t*u^2 + 2*L*p^3*q*t*u^2 - p^4*q*t*u^2 + 2*L^2*p^2*r*t*u^2 - 4*L*p^3*r*t*u^2 + 2*p^4*r*t*u^2 - 2*L^2*p*q*t^2*u^2 + 3*L*p^2*q*t^2*u^2 

### The explicit expression of $\Theta_2^+$

$$ \Theta_2^+ = \frac{q(\eta+1)}{2(\lambda-p)}\pmatrix{1 & 0\\ 0 & -1}\frac{dx}{y}.$$

## Final version of the Apparent Map

In [45]:
Nabla = c0*Nabla0plus + c1*Theta1plus + c2*Theta2plus
Nabla = Nabla.simplify_rational().factor()

In [46]:
vect = Nabla*vector((1,1))
APP = (vect[0] - vect[1]).simplify_rational().numerator().factor()

In [47]:
print APP

-(L^2*c1*p*r*u - L*c1*p^2*r*u - L^2*c0*q*t*u - L^2*c1*q*t*u - L*c2*p*q*t*u + L^2*c0*r*t*u - L*c0*p*r*t*u + L*c2*p*r*t*u - c2*p^2*r*t*u + L*c0*q*t^2*u + L*c1*q*t^2*u + c2*p*q*t^2*u + L^2*c0*q*u*x + L^2*c1*q*u*x + L*c2*p*q*u*x - L^2*c0*r*u*x - L^2*c1*r*u*x + L*c0*p*r*u*x - L*c2*p*r*u*x + c1*p^2*r*u*x + c2*p^2*r*u*x + L*c2*q*t*u*x - c2*p*q*t*u*x - L*c0*r*t*u*x - L*c2*r*t*u*x + c0*p*r*t*u*x + c2*p*r*t*u*x - c0*q*t^2*u*x - c1*q*t^2*u*x - c2*q*t^2*u*x - L*c0*q*u*x^2 - L*c1*q*u*x^2 - L*c2*q*u*x^2 + L*c0*r*u*x^2 + L*c1*r*u*x^2 + L*c2*r*u*x^2 - c0*p*r*u*x^2 - c1*p*r*u*x^2 - c2*p*r*u*x^2 + c0*q*t*u*x^2 + c1*q*t*u*x^2 + c2*q*t*u*x^2 - L^2*c1*p*r + L*c1*p^2*r - L^2*c0*q*t - L^2*c1*q*t - L*c2*p*q*t - L^2*c0*r*t + L*c0*p*r*t - L*c2*p*r*t + c2*p^2*r*t + L*c0*q*t^2 + L*c1*q*t^2 + c2*p*q*t^2 + L^2*c0*q*x + L^2*c1*q*x + L*c2*p*q*x + L^2*c0*r*x + L^2*c1*r*x - L*c0*p*r*x + L*c2*p*r*x - c1*p^2*r*x - c2*p^2*r*x + L*c2*q*t*x - c2*p*q*t*x + L*c0*r*t*x + L*c2*r*t*x - c0*p*r*t*x - c2*p*r*t*x - c0*q*t^2*x - c1*q

The above polynomial can be written as:

$$ c_0A_0(x-t)(x-\lambda) + c_1A_1(x-\lambda) + c_2A_2(x-t),$$

with $A_0$ independent of $x$, and $A_1,A_2$ linear polynomials on $x$. We now verify that the linear polynomials $A_i/A_0$ are $x-\mu_i$.

In [48]:
APP0 = diff(APP, c0).simplify_rational().factor()
APP1 = diff(APP, c1).simplify_rational().factor()
APP2 = diff(APP, c2).simplify_rational().factor()

In [49]:
APP0

(L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)*(L - x)*(n + 1)*q*(t - x)

In [50]:
APP1

-(L*p*r*u - p^2*r*u - L*q*t*u + q*t^2*u + L*q*u*x - L*r*u*x + p*r*u*x - q*t*u*x - L*p*r + p^2*r - L*q*t + q*t^2 + L*q*x + L*r*x - p*r*x - q*t*x)*(L - x)*(n + 1)*q

In [51]:
APP2

(L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)*(n + 1)*(p - x)*q*(t - x)

In [52]:
solve(APP1, x)

[x == (L*q*t - q*t^2 + (L*p - p^2)*r + (L*q*t - q*t^2 - (L*p - p^2)*r)*u)/(L*q + (L - p)*r - q*t + (L*q - (L - p)*r - q*t)*u), x == L]

In [53]:
W = (L*q*t - q*t^2 + (L*p - p^2)*r + (L*q*t - q*t^2 - (L*p - p^2)*r)*u)/(L*q + (L - p)*r - q*t + (L*q - (L - p)*r - q*t)*u)
W = W.simplify_rational().factor()
W

-(L*p*r*u - p^2*r*u - L*q*t*u + q*t^2*u - L*p*r + p^2*r - L*q*t + q*t^2)/(L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)

Up to a scalar factor (given by $A_0$), the apparent map is given by:

$$\operatorname{App}(\nabla) = (x-t)(x-\lambda) + c_1(x-\lambda)(x-W) + c_2(x-t)(x-p),$$

where $W$ is the second apparent singularity of $\Theta_1$ and is given by:

$$ W = -\frac{
    (qt(\lambda-t) - rp(\lambda-p))u + qt(\lambda-t) + rp(\lambda-p)
}{
    (q(\lambda-t) + r(\lambda-p))u + q(\lambda-t) - r(\lambda-p)
}. $$

We now see that the expression for $W$ is invariant under the substitution $(u,p,q)\mapsto(1/u,p,-q)$.

*Remark:* This formula for $W$ computes $\mu_1$ which in Loray-Saito is defined to be the second apparent singularity of $\Theta_1$. Also, we see that $\mu_2$, the second apparent singularity of $\Theta_2$, is exactly p.

In [54]:
# Check that the apparent map is as clamied above:
APP_final = c0*(x-t)*(x-L) + c1*(x-L)*(x-W) + c2*(x-t)*(x-p)
rescale_factor = (APP0/(x-t)/(x-L)).simplify_rational().factor()
print (APP_final*rescale_factor).simplify_rational().factor() - APP

0


In [55]:
APP.subs([c0==0, c2==0, x==W]).simplify_rational()

0

## Permuting the roles of $p_1,p_2$

Permuting the numbering of $p_1,p_2$ and then applying $z\mapsto z^{-1}$ transforms a connection into an isomorphic one. In our family, this corresponds to the transformation:

$M(u,p,q) \longmapsto BM(u^{-1}, p, -q)B^{-1} $,

where $B$ is obtained from the identity matrix by permuting the two columns.

In [56]:
B = matrix([(0,1), (1,0)])

In [57]:
# Nabla0plus is invariant
(Nabla0plus - B*Nabla0plus.subs([q==-q, u==1/u])*B).simplify_rational().factor() == 0

True

In [58]:
# Theta1plus is invariant
(Theta1plus - B*Theta1plus.subs([q==-q, u==1/u])*B).simplify_rational().factor() == 0

True

In [59]:
# Theta2 is invariant
(Theta2plus - B*Theta2plus.subs([q==-q, u==1/u])*B).simplify_rational().factor() == 0

True

# The sections $\nabla_0^-$ and $\Theta_1^-$

## Apparent map with respect to the section $\{z=-1\}$

In a similar way, we can define an apparent map with respect to the subbundle $z=-1$, and use it to define a connection $\nabla_0^-$

In [60]:
# From the expression
DM(M)

<IPython.core.display.Math object>

It follows that

$\operatorname{App}^-(x,y) = 2(a_2 - b_2)x^2 + 2(a_1 - b_1)x + 2(a_0 - b_0).$

Now we compute an explicit formula:

In [61]:
vect = (M.subs(Sols)).simplify_rational()*vector((1,-1))
APPminus = (vect[0] + vect[1]).simplify_rational().numerator().factor()

In [62]:
# Correct value of b_3 to define nabla0-
B3minus = -1/2*(p - t)*n

In [63]:
A2minus = a_2.subs(solve(APPminus.subs([x == L, b_3 == B3minus]), a_2))
A2minus = A2minus.simplify_rational().factor()
A2minus

1/4*(n*p^3*q*u + 2*L*n*p^2*r*u - 2*n*p^3*r*u - 2*L*n*p*q*t*u + n*p^2*q*t*u + n*p^3*q - 2*L*n*p^2*r + 2*n*p^3*r - 2*L*n*p*q*t + n*p^2*q*t + L*n*p*q*u - 2*n*p^2*q*u + 2*p^3*q*u - 2*L*n*p*r*u + 2*n*p^2*r*u + L*n*q*t*u - 2*p^2*q*t*u + L*n*p*q - 2*n*p^2*q + 2*p^3*q + 2*L*n*p*r - 2*n*p^2*r + L*n*q*t - 2*p^2*q*t - 2*p^2*q*u + 2*p*q*t*u - 2*p^2*q + 2*p*q*t)/((L - p)*(p - t)*(p - 1)*p*(u + 1))

In [64]:
# Verify that these choices of a_2, b_3 are the right ones:
APPminus.subs([a_2 == A2minus, b_3 == B3minus]).factor()

(L - x)*(n + 1)*(p - t)^2*(p - 1)*p*q*(t - x)*(u + 1)*(u - 1)

**Note:** The values of `A2plus` and `A2minus` are almost identical. Indeed, they differ only by a substitution $u\mapsto -u$.

In [65]:
(A2plus.subs(u==-u) - A2minus).simplify_rational()

0

This is because $\nabla_0^+$ and $\nabla_0^-$ correspond to each other after the transformation $z\mapsto -z$.

This transformation acts on connections as:

$$ M(u,p,q) = CM(-u,p,q)C^{-1},$$

where $C$ is the matrix $\operatorname{diag}(\{1,-1\})$.

### Definition of $\nabla_0^-$

In [66]:
nabla0minus = solve(Eq + [b_3 - B3minus, a_2 - A2minus], parameters)[0]
Nabla0minus = M.subs(nabla0minus).simplify_rational().factor()

Check that $\nabla_0^-$ is indeed $\nabla_0^+$ after the above transformation.

In [67]:
C = matrix([(-1,0), (0,1)])

(Nabla0minus - C*Nabla0plus.subs(u==-u)*C).simplify_rational().factor() == 0

True

### Definition of $\Theta_1^-$

For convenience we will just recover $\Theta_1^-$ from $\Theta_1^+$ using the transformation $C$.

In [68]:
Theta1minus = (C*Theta1plus.subs(u==-u)*C).simplify_rational().factor()

### Invariance of $\Theta_2$

In [69]:
(Theta2plus - C*Theta2plus.subs(u==-u)*C).simplify_rational().factor() == 0

True

In [70]:
Theta2 = Theta2plus

We don't need to define + and - versions of $\Theta_2$, since the first expression is already invariant.

# A family of connections $\nabla_0 + c_1\Theta_1 + c_2\Theta_2$ that is invariant under all automorphisms

## The Lagrangian section $\nabla_0 = \frac{1}{2}(\nabla_0^+ + \nabla_0^-)$

Note that $\nabla_0$ is given by the parameters:

$a_2 = \frac{1}{2}(A_2^+ + A_2^-)$,  
$b_3 = 0$.

In [71]:
A2 = (1/2)*(A2plus + A2minus).simplify_rational().factor()
B2 = 0

In [72]:
nabla0 = solve(Eq + [b_3, a_2 - A2], parameters)[0]
Nabla0 = M.subs(nabla0).simplify_rational().factor()

In [73]:
# The values of the parameters defining Nabla0 are not too complicated...
for v in parameters:
    if v.subs(nabla0) == 0:
        print '{} = 0'.format(v), '\n'
    else:
        print '{} ='.format(v), v.subs(nabla0).simplify_rational().factor(), '\n'

a_0 = 1/4*(2*L*n*p^3*r*u^2 - 2*n*p^4*r*u^2 + n*p^3*q*t*u^2 - 2*L*n*p*q*t^2*u^2 + n*p^2*q*t^2*u^2 - 2*L*n*p^2*r*u^2 + 2*n*p^3*r*u^2 + L*n*p*q*t*u^2 + 2*L*p^2*q*t*u^2 - 2*n*p^2*q*t*u^2 + L*n*q*t^2*u^2 - 2*L*p*q*t^2*u^2 + 2*L*n*p^3*r - 2*n*p^4*r - n*p^3*q*t + 2*L*n*p*q*t^2 - n*p^2*q*t^2 - 2*L*p*q*t*u^2 + 2*L*q*t^2*u^2 - 2*L*n*p^2*r + 2*n*p^3*r - L*n*p*q*t - 2*L*p^2*q*t + 2*n*p^2*q*t - L*n*q*t^2 + 2*L*p*q*t^2 + 2*L*p*q*t - 2*L*q*t^2)/((L - p)*(p - t)*(p - 1)*(u + 1)*(u - 1)) 

a_1 = -1/4*(n*p^4*q*u^2 + 4*L*n*p^3*r*u^2 - 4*n*p^4*r*u^2 - 2*L*n*p^2*q*t*u^2 + 2*n*p^3*q*t*u^2 - 2*L*n*p*q*t^2*u^2 + n*p^2*q*t^2*u^2 + L*n*p^2*q*u^2 + 2*L*p^3*q*u^2 - 2*n*p^3*q*u^2 - 4*L*n*p^2*r*u^2 + 4*n*p^3*r*u^2 + 2*L*n*p*q*t*u^2 - 2*L*p^2*q*t*u^2 - 2*n*p^2*q*t*u^2 + 2*p^3*q*t*u^2 + L*n*q*t^2*u^2 - 2*p^2*q*t^2*u^2 - n*p^4*q + 4*L*n*p^3*r - 4*n*p^4*r + 2*L*n*p^2*q*t - 2*n*p^3*q*t + 2*L*n*p*q*t^2 - n*p^2*q*t^2 - 2*L*p^2*q*u^2 + 2*L*p*q*t*u^2 - 2*p^2*q*t*u^2 + 2*p*q*t^2*u^2 - L*n*p^2*q - 2*L*p^3*q + 2*n*p^3*q - 4*L*

In [74]:
# Residues of Nabla0
for k in range(4):
    print 'Residue at {}:'.format(Sing[k])
    residue = Res[k].subs(nabla0).simplify_rational().factor()
    DM(residue)
    print 'Eigenvectors:', residue.eigenvectors_right()
    print '\n'

Residue at [x == p, y == q]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2, [(1, 0)], 1), (1/2, [(0, 1)], 1)]


Residue at [x == p, y == -q]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2, [(0, 1)], 1), (1/2, [(1, 0)], 1)]


Residue at [x == t, y == r]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2*n, [(1, 1/u)], 1), (1/2*n, [(1, u)], 1)]


Residue at [x == t, y == -r]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2*n, [(1, u)], 1), (1/2*n, [(1, 1/u)], 1)]




**Question:** How to characterize $\nabla_0$ in a geometric way?? The apparent maps don't seem to give a lot of information...

### Invariance of $\nabla_0$

By construction, $\nabla_0$ is invariant under $z\mapsto-z$, but what about the other involution (permuting the roles of p1 and p2)?

In [75]:
# Recall B is the permutation matrix B = matrix([(0,1), (1,0)])
(Nabla0 - B*Nabla0.subs([q==-q, u==1/u])*B).simplify_rational().factor() == 0

True

In [76]:
# Additional symmetries:
print (Nabla0 - Nabla0.subs([r==-r, u==1/u])).simplify_rational().factor() == 0
print (Nabla0 - (-1)*Nabla0.subs([q==-q, r==-r])).simplify_rational().factor() == 0

True
True


## Definition of $\Theta_1 = \frac{1}{2}(\Theta_1^+ + \Theta_1^-)$

Note that $\Theta_2$ is invarian under the transformation $z \mapsto -z$, however, $\Theta_1$ isn't. To define an invariant Higgs field we'll take the average of the + and the - versions of it (as we did for $\nabla_0$).

In [77]:
Theta1 = (1/2)*(Theta1plus + Theta1minus).simplify_rational().factor()

In [78]:
Theta2 = Theta2plus

Now we have a Universal Family spanned by the elements $\nabla_0$, $\Theta_1$, and $\Theta_2$, which are invariant under both involutions.  
To confirm the invariance:

In [79]:
print (Nabla0 - C*Nabla0.subs([u==-u])*C) == 0
print (Nabla0 - B*Nabla0.subs([q==-q, u==1/u])*B).simplify_rational().factor() == 0, '\n'

print (Theta1 - C*Theta1.subs([u==-u])*C) == 0
print (Theta1 - B*Theta1.subs([q==-q, u==1/u])*B).simplify_rational().factor() == 0, '\n'

print (Theta2 - C*Theta2.subs([u==-u])*C) == 0
print (Theta2 - B*Theta2.subs([q==-q, u==1/u])*B).simplify_rational().factor() == 0

True
True 

True
True 

True
True


# The quotient $u\sim -u$

**Remark:** From the expression $\Theta_1^-(p,q,u) = C\Theta_1^+(p,q,-u)C$, we deduce that $\Theta_1^-(p,q,u)$ is characterized by having parabolic structure $(p,q,u)$ and tangencies with the horizontal $\{z = -1\}$ exactly at the points $t$ and $\mu_1(p,q,-u)$. The minus sign inside $\mu_1(p,q,-u)$ is important!

First of all, we'll need to write $\nabla_0^-$ and $\Theta_1^-$ as a linear combination of $\nabla_0^+$, $\Theta_1^+$ and $\Theta_2$.

In [80]:
var('C01 C02 C11 C12')

(C01, C02, C11, C12)

In [81]:
conjugacy_matrix = matrix([(1, C01, C02), (0, C11, C12), (0, 0, 1)]).transpose(); conjugacy_matrix

[  1   0   0]
[C01 C11   0]
[C02 C12   1]

In [82]:
# conjugacy^2 must be the identity
solve([(conjugacy_matrix^2 - identity_matrix(3))[i][j] for i in range(3) for j in range(3)], [C01, C02, C11, C12])

[[C01 == r1, C02 == -1/2*r1*r2, C11 == -1, C12 == r2], [C01 == r3, C02 == 0, C11 == -1, C12 == 0], [C01 == 0, C02 == 0, C11 == 1, C12 == 0]]

**Warning:** The above are only valid if the coefficients Cij are constants, but they do depend on u, so we need to switch from u to -u each time we conjugate.

Computations for $\nabla_0^-$

In [83]:
EqN0 = C*Nabla0plus*C - (Nabla0plus + c1*Theta1plus + c2*Theta2plus).subs(u==-u)

In [84]:
c01 = c1.subs(solve(diff(EqN0[0][1], y).simplify_rational().factor(), c1))
c01 = c01.simplify_rational().factor()
print c01

-2*(L*q*u - L*r*u + p*r*u - q*t*u - L*q - L*r + p*r + q*t)*n/((L - t)*(n + 1)*q*(u - 1))


In [85]:
c02 = c2.subs(solve(EqN0[0][0].subs(c1==c01).subs(x==0).simplify_rational(), c2))
c02 = c02.simplify_rational().factor()
print c02

(L*p^3*q*u + 2*L^2*p^2*r*u - 4*L*p^3*r*u + 2*p^4*r*u - 2*L^2*p*q*t*u + L*p^2*q*t*u - p^3*q*t*u + 2*L*p*q*t^2*u - p^2*q*t^2*u - L*p^3*q + 2*L^2*p^2*r - 4*L*p^3*r + 2*p^4*r + 2*L^2*p*q*t - L*p^2*q*t + p^3*q*t - 2*L*p*q*t^2 + p^2*q*t^2 + L^2*p*q*u - 2*L*p^2*q*u - 2*L^2*p*r*u + 4*L*p^2*r*u - 2*p^3*r*u + L^2*q*t*u - L*p*q*t*u + 2*p^2*q*t*u - L*q*t^2*u - L^2*p*q + 2*L*p^2*q - 2*L^2*p*r + 4*L*p^2*r - 2*p^3*r - L^2*q*t + L*p*q*t - 2*p^2*q*t + L*q*t^2)*n/((L - t)*(n + 1)*(p - t)*(p - 1)*p*q*(u - 1))


In [86]:
EqN0.subs([c1==c01, c2==c02]).simplify_rational()

[0 0]
[0 0]

Computations for $\Theta_1^-$

In [87]:
EqT1 = C*Theta1plus*C - (c1*Theta1plus + c2*Theta2plus).subs(u==-u)

In [88]:
c11 = c1.subs(solve(diff(EqT1[0][1], y).simplify_rational().factor(), c1))
c11 = c11.simplify_rational().factor()
print c11

-(L*q*u - L*r*u + p*r*u - q*t*u - L*q - L*r + p*r + q*t)*(u + 1)/((L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)*(u - 1))


In [89]:
c12 = c2.subs(solve(EqT1[0][0].subs(c1==c11).subs(x==0).simplify_rational(), c2))
c12 = c12.simplify_rational().factor()
print c12

(L*p^3*q*u^2 + 2*L^2*p^2*r*u^2 - 4*L*p^3*r*u^2 + 2*p^4*r*u^2 - 2*L^2*p*q*t*u^2 + L*p^2*q*t*u^2 - p^3*q*t*u^2 + 2*L*p*q*t^2*u^2 - p^2*q*t^2*u^2 + L^2*p*q*u^2 - 2*L*p^2*q*u^2 - 2*L^2*p*r*u^2 + 4*L*p^2*r*u^2 - 2*p^3*r*u^2 + L^2*q*t*u^2 - L*p*q*t*u^2 + 2*p^2*q*t*u^2 - L*q*t^2*u^2 - L*p^3*q + 2*L^2*p^2*r - 4*L*p^3*r + 2*p^4*r + 2*L^2*p*q*t - L*p^2*q*t + p^3*q*t - 2*L*p*q*t^2 + p^2*q*t^2 - L^2*p*q + 2*L*p^2*q - 2*L^2*p*r + 4*L*p^2*r - 2*p^3*r - L^2*q*t + L*p*q*t - 2*p^2*q*t + L*q*t^2)/((L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)*(p - t)*(p - 1)*p*(u - 1))


In [90]:
EqT1.subs([c1==c11, c2==c12]).simplify_rational()

[0 0]
[0 0]

In [91]:
conjugacy_matrix = conjugacy_matrix.subs([C01==c01, C02==c02, C11==c11, C12==c12])

### The invariant universal family

In order to change base from $(\nabla_0^+, \Theta_1^+, \Theta_2)$ to $(\nabla_0, \Theta_1, \Theta_2)$, we just need to take the average of the identity and the conjugacy matrix.

**Remark:** The main feature of this basis is that if a connection $\nabla$ is specified by the parameters $(p,q,u,c_1,c_2)$, then the only other connection with the same values of $(p,q)$ that is isomorphic to $\nabla$ is the connection $\tilde{\nabla}$ specified by the parameters $(p,q,-u,c_1,c_2)$.

In [92]:
change_base = (1/2)*(identity_matrix(3) + conjugacy_matrix).simplify_rational().factor()

In [93]:
inverse_change_base = (~change_base).simplify_rational().factor()

In [94]:
# Sanity check
(change_base - change_base.subs([u==1/u, q==-q])).simplify_rational() == 0

True

Finally, we can describe the map $\Phi$ between our families of connections over $\mathbb{P}^1$ and over $C$ as follows:

$$ (l, c, (c_1, c_2)) \longmapsto (p, q, u, \operatorname{ICB}(1, c_1, c_2)), $$

where ICB is the inverse change of base matrix (which depends on p,q,u).

# The pull-back map $\Phi$ in coordinates

Let $\mu_1, \mu_2$ be the (second) apparent singularities of $\Theta_1^+$ and $\Theta_1^-$ respectively. Then, according to Loray-Saito we have the following:

$$\mu_1 = t\frac{l-1}{l-t}, \qquad \mu_2 = \lambda\frac{c-1}{c-\lambda}. $$

**Remark:** Note that $\mu_2 = p$. We have also computed $\mu_1$ in terms of our variables $u,p,q$ (we called it $W$ before) as:

$$ \mu_1 = W = -\frac{
    (qt(\lambda-t) - rp(\lambda-p))u + qt(\lambda-t) + rp(\lambda-p)
}{
    (q(\lambda-t) + r(\lambda-p))u + q(\lambda-t) - r(\lambda-p)
}. $$

We can use these equations to solve for $l,c$ in terms of $u,p$ or viceversa.

### Inverse image of $(u,p,q)$

In [95]:
var('l c')  # l is parabolic over t, and c is parabolic over L
dict['c'] = c; dict['l'] = l

In [96]:
# Solve mu1 = W, mu2 = p for c,l:
Sols_lc = solve([t*(l-1)/(l-t) == W, L*(c-1)/(c-L) == p], [c,l])[0]

In [97]:
print 'c =', c.subs(Sols_lc).simplify_rational().factor(), '\n'
print 'l =', l.subs(Sols_lc).simplify_rational().factor()

c = -L*(p - 1)/(L - p) 

l = (L*p*r*u - p^2*r*u - L*q*t*u + q*t^2*u - L*p*r + p^2*r - L*q*t + q*t^2 + L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)*t/((L - p)*(p - t)*r*(u - 1))


Let's simplify the expression for `l` a little...

In [98]:
simple_l = simplify_qr(l.subs(Sols_lc)*r^2)/r2; simple_l

(L*p*t*u - p^2*t*u - L*p*t + p^2*t + q*r*u - L*t*u + p*t*u + q*r + L*t - p*t)/((L - p)*(p - t)*(u - 1))

In [99]:
# Invariance under the first symmetry
simple_l.subs([u==1/u, q==-q]).simplify_rational().factor() - simple_l

0

In [100]:
simple_l.subs(u==-u).simplify_rational().factor()  # conjugate value of l (ie both define the same value of w)

(L*p*t*u - p^2*t*u + L*p*t - p^2*t + q*r*u - L*t*u + p*t*u - q*r - L*t + p*t)/((L - p)*(p - t)*(u + 1))

### Direct image of $(l,c)$

We can also use the formulas for the $\mu_i$ to solve for $u,p,q$ given $l,c$

In [101]:
# Solve mu1 = W, mu2 = p for u,p:
Sols_up = solve([t*(l-1)/(l-t) == W, L*(c-1)/(c-L) == p], [u,p])[0]

In [102]:
print 'u =', u.subs(Sols_up).simplify_rational().factor(), '\n'
print 'p =', p.subs(Sols_up).simplify_rational().factor()

u = (L^3*c*l*r - L^3*c*r*t + L^3*l*r*t - L^2*c*l*r*t + L^3*q*t^2 - 2*L^2*c*q*t^2 + L*c^2*q*t^2 - L^2*q*t^3 + 2*L*c*q*t^3 - c^2*q*t^3 - L^3*l*r - L^2*c*l*r - L^3*q*t + 2*L^2*c*q*t - L*c^2*q*t + 2*L^2*c*r*t - L^2*l*r*t + L*c*l*r*t + L^2*q*t^2 - 2*L*c*q*t^2 + c^2*q*t^2 + L^2*l*r - L*c*r*t)/(L^3*c*l*r - L^3*c*r*t + L^3*l*r*t - L^2*c*l*r*t - L^3*q*t^2 + 2*L^2*c*q*t^2 - L*c^2*q*t^2 + L^2*q*t^3 - 2*L*c*q*t^3 + c^2*q*t^3 - L^3*l*r - L^2*c*l*r + L^3*q*t - 2*L^2*c*q*t + L*c^2*q*t + 2*L^2*c*r*t - L^2*l*r*t + L*c*l*r*t - L^2*q*t^2 + 2*L*c*q*t^2 - c^2*q*t^2 + L^2*l*r - L*c*r*t) 

p = -L*(c - 1)/(L - c)


### Transform $\nabla_0$ and $\Theta_1$ into $(l,c)$ variables

In [103]:
var('v')  # represents sqrt(c*(c-1)*(c-L))

v

We have the following transformation rules:
```python
v == q*L*(L-1)/(L-p)^2
q == v*L*(L-1)/(L-c)^2
```

In [104]:
to_cl_vars = [
    p == -L*(c - 1)/(L - c),
    q == v*L*(L-1)/(L-c)^2,
    u == (L*c*l*r - L*c*r*t + L*l*r*t - c*l*r*t + L*t^2*v - t^3*v - L*l*r + c*r*t - L*t*v + t^2*v)/(L*c*l*r - L*c*r*t + L*l*r*t - c*l*r*t - L*t^2*v + t^3*v - L*l*r + c*r*t + L*t*v - t^2*v)
]

In [105]:
v2 = P.subs(x==c)
dict['v'] = v
dict['v2'] = v2

def simplify_vr(expr):
    """Convert expr to a string and replace powers of v and r"""
    global dict
    expr = expr.simplify_rational()
    num = expr.numerator().expand()
    str_num = str(num)
    str_num = re.sub('v\^2', 'v2', str_num)
    str_num = re.sub('v\^3', 'v*v2', str_num)
    str_num = re.sub('v\^4', 'v2^2', str_num)
    str_num = re.sub('r\^2', 'r2', str_num)
    str_num = re.sub('r\^3', 'r*r2', str_num)
    str_num = re.sub('r\^4', 'r2^2', str_num)
    
    den = expr.denominator().expand()
    str_den = str(den)
    str_den = re.sub('v\^2', 'v2', str_den)
    str_den = re.sub('v\^3', 'v*v2', str_den)
    str_den = re.sub('v\^4', 'v2^2', str_den)
    str_den = re.sub('r\^2', 'r2', str_den)
    str_den = re.sub('r\^3', 'r*r2', str_den)
    str_den = re.sub('r\^4', 'r2^2', str_den)
    
    new_expr = sage_eval(str_num, locals=dict)/sage_eval(str_den, locals=dict)
    new_expr = new_expr.simplify_rational()
    if new_expr == 0:
        return new_expr
    else:
        return new_expr.factor()

In [106]:
N00 = 1/4*(L^3*c^3*l^2*n - 2*L^3*c^3*l*n*t + L^2*c^4*l*n*t + L^3*c^2*l^2*n*t - L^2*c^3*l^2*n*t + 2*L^3*c^3*n*t^2 - 2*L^2*c^4*n*t^2 - 2*L^3*c^2*l*n*t^2 + 3*L^2*c^3*l*n*t^2 - L*c^4*l*n*t^2 - L^2*c^4*l*n*x + 2*L^3*c^2*l^2*n*x - 2*L^2*c^3*l^2*n*x + L^2*c^4*n*t*x - 2*L^3*c^2*l*n*t*x + 2*L^2*c^3*l*n*t*x + 2*L^3*c*l^2*n*t*x - 4*L^2*c^2*l^2*n*t*x + 2*L*c^3*l^2*n*t*x + 2*L^3*c^2*n*t^2*x - 5*L^2*c^3*n*t^2*x + 3*L*c^4*n*t^2*x - 2*L^3*c*l*n*t^2*x + 5*L^2*c^2*l*n*t^2*x - 4*L*c^3*l*n*t^2*x + c^4*l*n*t^2*x - L^2*c^3*l*n*x^2 + L*c^4*l*n*x^2 + L^3*c*l^2*n*x^2 - 2*L^2*c^2*l^2*n*x^2 + L*c^3*l^2*n*x^2 + L^2*c^3*n*t*x^2 - L*c^4*n*t*x^2 - L^2*c^2*l*n*t*x^2 + 2*L*c^3*l*n*t*x^2 - c^4*l*n*t*x^2 + L^3*l^2*n*t*x^2 - 3*L^2*c*l^2*n*t*x^2 + 3*L*c^2*l^2*n*t*x^2 - c^3*l^2*n*t*x^2 - L^2*c^2*n*t^2*x^2 + 2*L*c^3*n*t^2*x^2 - c^4*n*t^2*x^2 - 3*L^3*c^2*l^2*n - 2*L^3*c^3*l*t + 2*L^2*c^4*l*t - L^3*c^3*n*t + L^2*c^4*n*t + 5*L^3*c^2*l*n*t - 2*L^2*c^3*l*n*t - 2*L^3*c*l^2*n*t + 2*L^2*c^2*l^2*n*t + 2*L^3*c^3*t^2 - 2*L^2*c^4*t^2 - 2*L^3*c^2*l*t^2 + 4*L^2*c^3*l*t^2 - 2*L*c^4*l*t^2 - 3*L^3*c^2*n*t^2 + 2*L^2*c^3*n*t^2 + L*c^4*n*t^2 + 3*L^3*c*l*n*t^2 - 4*L^2*c^2*l*n*t^2 + L*c^3*l*n*t^2 + 2*L^3*c^3*l*x - 2*L^2*c^4*l*x - L^3*c^2*l*n*x + 4*L^2*c^3*l*n*x - 4*L^3*c*l^2*n*x + 4*L^2*c^2*l^2*n*x - 2*L^3*c^3*t*x + 2*L^2*c^4*t*x + 2*L^3*c^2*l*t*x - 2*L^2*c^3*l*t*x - L^3*c^2*n*t*x + L^2*c^3*n*t*x - 3*L*c^4*n*t*x + 2*L^3*c*l*n*t*x - 2*L*c^3*l*n*t*x - 2*L^3*l^2*n*t*x + 4*L^2*c*l^2*n*t*x - 2*L*c^2*l^2*n*t*x - 2*L^2*c^3*t^2*x + 2*L*c^4*t^2*x + 2*L^2*c^2*l*t^2*x - 4*L*c^3*l*t^2*x + 2*c^4*l*t^2*x - L^3*c*n*t^2*x + 3*L^2*c^2*n*t^2*x - L*c^3*n*t^2*x - c^4*n*t^2*x + L^3*l*n*t^2*x - 2*L^2*c*l*n*t^2*x + L*c^2*l*n*t^2*x - 2*L^2*c^3*l*x^2 + 2*L*c^4*l*x^2 - L^3*c*l*n*x^2 + 4*L^2*c^2*l*n*x^2 - 3*L*c^3*l*n*x^2 - L^3*l^2*n*x^2 + 2*L^2*c*l^2*n*x^2 - L*c^2*l^2*n*x^2 + 2*L^2*c^3*t*x^2 - 2*L*c^4*t*x^2 - 2*L^2*c^2*l*t*x^2 + 4*L*c^3*l*t*x^2 - 2*c^4*l*t*x^2 - 2*L*c^3*n*t*x^2 + 2*c^4*n*t*x^2 - L^3*l*n*t*x^2 + 4*L^2*c*l*n*t*x^2 - 5*L*c^2*l*n*t*x^2 + 2*c^3*l*n*t*x^2 + L^2*c*n*t^2*x^2 - 2*L*c^2*n*t^2*x^2 + c^3*n*t^2*x^2 + 3*L^3*c*l^2*n + 4*L^3*c^2*l*t - 4*L^2*c^3*l*t + 2*L^3*c^2*n*t - 2*L^2*c^3*n*t - 4*L^3*c*l*n*t + L^2*c^2*l*n*t + L^3*l^2*n*t - L^2*c*l^2*n*t - 2*L^3*c^2*t^2 + 2*L*c^4*t^2 + 2*L^3*c*l*t^2 - 4*L^2*c^2*l*t^2 + 2*L*c^3*l*t^2 + L^3*c*n*t^2 - L*c^3*n*t^2 - L^3*l*n*t^2 + L^2*c*l*n*t^2 - 4*L^3*c^2*l*x + 4*L^2*c^3*l*x + 2*L^3*c*l*n*x - 5*L^2*c^2*l*n*x + 2*L^3*l^2*n*x - 2*L^2*c*l^2*n*x + 2*L^3*c^2*t*x - 2*L*c^4*t*x - 2*L^3*c*l*t*x + 2*L*c^3*l*t*x + L^3*c*n*t*x - 3*L^2*c^2*n*t*x + 5*L*c^3*n*t*x - 2*L^2*c*l*n*t*x + 2*L*c^2*l*n*t*x + 2*L^2*c^2*t^2*x - 2*c^4*t^2*x - 2*L^2*c*l*t^2*x + 4*L*c^2*l*t^2*x - 2*c^3*l*t^2*x - L^2*c*n*t^2*x + L*c^2*n*t^2*x + 4*L^2*c^2*l*x^2 - 4*L*c^3*l*x^2 + L^3*l*n*x^2 - 3*L^2*c*l*n*x^2 + 2*L*c^2*l*n*x^2 - 2*L^2*c^2*t*x^2 + 2*c^4*t*x^2 + 2*L^2*c*l*t*x^2 - 4*L*c^2*l*t*x^2 + 2*c^3*l*t*x^2 - L^2*c*n*t*x^2 + 3*L*c^2*n*t*x^2 - 2*c^3*n*t*x^2 - L^3*l^2*n - 2*L^3*c*l*t + 2*L^2*c^2*l*t - L^3*c*n*t + L^2*c^2*n*t + L^3*l*n*t + 2*L^2*c^2*t^2 - 2*L*c^3*t^2 + 2*L^3*c*l*x - 2*L^2*c^2*l*x - L^3*l*n*x + 2*L^2*c*l*n*x - 2*L^2*c^2*t*x + 2*L*c^3*t*x + 2*L^2*c*l*t*x - 2*L*c^2*l*t*x + L^2*c*n*t*x - 2*L*c^2*n*t*x - 2*L*c^2*t^2*x + 2*c^3*t^2*x - 2*L^2*c*l*x^2 + 2*L*c^2*l*x^2 + 2*L*c^2*t*x^2 - 2*c^3*t*x^2)/((L*c*l - L*c*t + L*l*t - c*l*t - L*l + c*t)*(L - c)*v)
print N00

1/4*(L^3*c^3*l^2*n - 2*L^3*c^3*l*n*t + L^2*c^4*l*n*t + L^3*c^2*l^2*n*t - L^2*c^3*l^2*n*t + 2*L^3*c^3*n*t^2 - 2*L^2*c^4*n*t^2 - 2*L^3*c^2*l*n*t^2 + 3*L^2*c^3*l*n*t^2 - L*c^4*l*n*t^2 - L^2*c^4*l*n*x + 2*L^3*c^2*l^2*n*x - 2*L^2*c^3*l^2*n*x + L^2*c^4*n*t*x - 2*L^3*c^2*l*n*t*x + 2*L^2*c^3*l*n*t*x + 2*L^3*c*l^2*n*t*x - 4*L^2*c^2*l^2*n*t*x + 2*L*c^3*l^2*n*t*x + 2*L^3*c^2*n*t^2*x - 5*L^2*c^3*n*t^2*x + 3*L*c^4*n*t^2*x - 2*L^3*c*l*n*t^2*x + 5*L^2*c^2*l*n*t^2*x - 4*L*c^3*l*n*t^2*x + c^4*l*n*t^2*x - L^2*c^3*l*n*x^2 + L*c^4*l*n*x^2 + L^3*c*l^2*n*x^2 - 2*L^2*c^2*l^2*n*x^2 + L*c^3*l^2*n*x^2 + L^2*c^3*n*t*x^2 - L*c^4*n*t*x^2 - L^2*c^2*l*n*t*x^2 + 2*L*c^3*l*n*t*x^2 - c^4*l*n*t*x^2 + L^3*l^2*n*t*x^2 - 3*L^2*c*l^2*n*t*x^2 + 3*L*c^2*l^2*n*t*x^2 - c^3*l^2*n*t*x^2 - L^2*c^2*n*t^2*x^2 + 2*L*c^3*n*t^2*x^2 - c^4*n*t^2*x^2 - 3*L^3*c^2*l^2*n - 2*L^3*c^3*l*t + 2*L^2*c^4*l*t - L^3*c^3*n*t + L^2*c^4*n*t + 5*L^3*c^2*l*n*t - 2*L^2*c^3*l*n*t - 2*L^3*c*l^2*n*t + 2*L^2*c^2*l^2*n*t + 2*L^3*c^3*t^2 - 2*L^2*c^4*t^2 - 2*L^3

In [107]:
N01 = -1/4*(L*c*l^2 - 2*L*c*l*t + L*l^2*t - c*l^2*t + c^2*t^2 - L*l^2 + L*c*t - c^2*t + 2*c*l*t - c*t^2)*(L*c + L*x - c*x - L)^2*n/((L*c*l - L*c*t + L*l*t - c*l*t - L*l + c*t)*(L - c)*v)
print N01

-1/4*(L*c*l^2 - 2*L*c*l*t + L*l^2*t - c*l^2*t + c^2*t^2 - L*l^2 + L*c*t - c^2*t + 2*c*l*t - c*t^2)*(L*c + L*x - c*x - L)^2*n/((L*c*l - L*c*t + L*l*t - c*l*t - L*l + c*t)*(L - c)*v)


In [108]:
T00 = -1/8*(L^2*c^2*l^4 - 2*L*c^4*l^2*t - 2*L^2*c^2*l^3*t + L*c^3*l^3*t + L^2*c*l^4*t - L*c^2*l^4*t + 4*L*c^4*l*t^2 + 2*L^2*c^2*l^2*t^2 - 5*L*c^3*l^2*t^2 + 2*c^4*l^2*t^2 - 2*L^2*c*l^3*t^2 + 3*L*c^2*l^3*t^2 - c^3*l^3*t^2 - 2*L*c^4*t^3 + 2*L*c^3*l*t^3 - c^4*l*t^3 + 2*L*c^4*l^2*x - 2*L^2*c^2*l^3*x - L*c^3*l^3*x + L^2*c*l^4*x - L*c^2*l^4*x - 4*L*c^4*l*t*x + 2*L^2*c^2*l^2*t*x + 7*L*c^3*l^2*t*x - 2*c^4*l^2*t*x - 2*L^2*c*l^3*t*x + L*c^2*l^3*t*x + c^3*l^3*t*x + L^2*l^4*t*x - 2*L*c*l^4*t*x + c^2*l^4*t*x + 2*L*c^4*t^2*x - 6*L*c^3*l*t^2*x + c^4*l*t^2*x + 2*L*c^2*l^2*t^2*x - 2*c^3*l^2*t^2*x + c^4*t^3*x - 2*L^2*c*l^4 + 4*L*c^3*l^2*t + 3*L^2*c*l^3*t - L*c^2*l^3*t - L^2*l^4*t + L*c*l^4*t - 2*L^2*c^2*l*t^2 - 3*L*c^3*l*t^2 - 3*c^4*l*t^2 + L^2*c*l^2*t^2 + c^3*l^2*t^2 + L^2*l^3*t^2 - L*c*l^3*t^2 + 3*L*c^3*t^3 + c^4*t^3 - 3*L*c^2*l*t^3 + c^3*l*t^3 + 2*L^2*c^2*l^2*x - 6*L*c^3*l^2*x + L^2*c*l^3*x + 5*L*c^2*l^3*x - L^2*l^4*x + L*c*l^4*x - 2*L^2*c^2*l*t*x + 7*L*c^3*l*t*x + 3*c^4*l*t*x + L^2*c*l^2*t*x - 16*L*c^2*l^2*t*x + c^3*l^2*t*x - L^2*l^3*t*x + 5*L*c*l^3*t*x - 4*c^2*l^3*t*x - L*c^3*t^2*x - 3*c^4*t^2*x + 7*L*c^2*l*t^2*x + 3*c^3*l*t^2*x - 2*L*c*l^2*t^2*x + 2*c^2*l^2*t^2*x - 2*c^3*t^3*x + L^2*l^4 - 2*L*c^2*l^2*t - L^2*l^3*t + L^2*c^2*t^2 - 2*L*c^3*t^2 + c^4*t^2 - L^2*c*l*t^2 + 5*L*c^2*l*t^2 - L*c*l^2*t^2 - L*c^2*t^3 - c^3*t^3 + L*c*l*t^3 - 2*L^2*c*l^2*x + 4*L*c^2*l^2*x + L^2*l^3*x - 4*L*c*l^3*x + L^2*c*l*t*x - L*c^2*l*t*x - 4*c^3*l*t*x + 3*L*c*l^2*t*x + 4*c^2*l^2*t*x - L*c^2*t^2*x + 3*c^3*t^2*x - L*c*l*t^2*x - 4*c^2*l*t^2*x + c^2*t^3*x)*(L*c + L*x - c*x - L)*(n + 1)/((L*c*l - L*c*t + L*l*t - c*l*t - L*l + c*t)*(L - c)*(c - l)*(l - t)*v)
print T00

-1/8*(L^2*c^2*l^4 - 2*L*c^4*l^2*t - 2*L^2*c^2*l^3*t + L*c^3*l^3*t + L^2*c*l^4*t - L*c^2*l^4*t + 4*L*c^4*l*t^2 + 2*L^2*c^2*l^2*t^2 - 5*L*c^3*l^2*t^2 + 2*c^4*l^2*t^2 - 2*L^2*c*l^3*t^2 + 3*L*c^2*l^3*t^2 - c^3*l^3*t^2 - 2*L*c^4*t^3 + 2*L*c^3*l*t^3 - c^4*l*t^3 + 2*L*c^4*l^2*x - 2*L^2*c^2*l^3*x - L*c^3*l^3*x + L^2*c*l^4*x - L*c^2*l^4*x - 4*L*c^4*l*t*x + 2*L^2*c^2*l^2*t*x + 7*L*c^3*l^2*t*x - 2*c^4*l^2*t*x - 2*L^2*c*l^3*t*x + L*c^2*l^3*t*x + c^3*l^3*t*x + L^2*l^4*t*x - 2*L*c*l^4*t*x + c^2*l^4*t*x + 2*L*c^4*t^2*x - 6*L*c^3*l*t^2*x + c^4*l*t^2*x + 2*L*c^2*l^2*t^2*x - 2*c^3*l^2*t^2*x + c^4*t^3*x - 2*L^2*c*l^4 + 4*L*c^3*l^2*t + 3*L^2*c*l^3*t - L*c^2*l^3*t - L^2*l^4*t + L*c*l^4*t - 2*L^2*c^2*l*t^2 - 3*L*c^3*l*t^2 - 3*c^4*l*t^2 + L^2*c*l^2*t^2 + c^3*l^2*t^2 + L^2*l^3*t^2 - L*c*l^3*t^2 + 3*L*c^3*t^3 + c^4*t^3 - 3*L*c^2*l*t^3 + c^3*l*t^3 + 2*L^2*c^2*l^2*x - 6*L*c^3*l^2*x + L^2*c*l^3*x + 5*L*c^2*l^3*x - L^2*l^4*x + L*c*l^4*x - 2*L^2*c^2*l*t*x + 7*L*c^3*l*t*x + 3*c^4*l*t*x + L^2*c*l^2*t*x - 16*L*c^2*l^2

In [109]:
T01 = 1/8*(L^2*c^2*l^2 - L*c^3*l*t - 2*L^2*c^2*l*x + L*c^3*l*x + 2*L^2*c*l^2*x - 2*L*c^2*l^2*x + 2*L^2*c^2*t*x - L*c^3*t*x - 2*L^2*c*l*t*x + L*c^2*l*t*x + c^3*l*t*x + L*c^2*l*x^2 - c^3*l*x^2 + L^2*l^2*x^2 - 2*L*c*l^2*x^2 + c^2*l^2*x^2 - L*c^2*t*x^2 + c^3*t*x^2 - 2*L^2*c*l^2 - L^2*c^2*t + L*c^3*t + L^2*c*l*t + L*c^2*l*t + L^2*c*l*x + L*c^2*l*x - 2*L^2*l^2*x + 2*L*c*l^2*x - L^2*c*t*x - c^3*t*x + L^2*l*t*x + L*c*l*t*x - 2*c^2*l*t*x - L^2*l*x^2 + L*c*l*x^2 + L*c*t*x^2 - c^2*t*x^2 + 2*L*c*l*v*y - 2*L*c*t*v*y + 2*L*l*t*v*y - 2*c*l*t*v*y + L^2*l^2 + L^2*c*t - L*c^2*t - L^2*l*t + L^2*l*x - 2*L*c*l*x - L*c*t*x + 2*c^2*t*x - 2*L*l*v*y + 2*c*t*v*y)*(L*c*l^2 - 2*L*c*l*t + L*l^2*t - c*l^2*t + c^2*t^2 - L*l^2 + L*c*t - c^2*t + 2*c*l*t - c*t^2)*(n + 1)/((L*c*l - L*c*t + L*l*t - c*l*t - L*l + c*t)*(L - c)*(c - l)*(l - t)*v) 
print T01

1/8*(L^2*c^2*l^2 - L*c^3*l*t - 2*L^2*c^2*l*x + L*c^3*l*x + 2*L^2*c*l^2*x - 2*L*c^2*l^2*x + 2*L^2*c^2*t*x - L*c^3*t*x - 2*L^2*c*l*t*x + L*c^2*l*t*x + c^3*l*t*x + L*c^2*l*x^2 - c^3*l*x^2 + L^2*l^2*x^2 - 2*L*c*l^2*x^2 + c^2*l^2*x^2 - L*c^2*t*x^2 + c^3*t*x^2 - 2*L^2*c*l^2 - L^2*c^2*t + L*c^3*t + L^2*c*l*t + L*c^2*l*t + L^2*c*l*x + L*c^2*l*x - 2*L^2*l^2*x + 2*L*c*l^2*x - L^2*c*t*x - c^3*t*x + L^2*l*t*x + L*c*l*t*x - 2*c^2*l*t*x - L^2*l*x^2 + L*c*l*x^2 + L*c*t*x^2 - c^2*t*x^2 + 2*L*c*l*v*y - 2*L*c*t*v*y + 2*L*l*t*v*y - 2*c*l*t*v*y + L^2*l^2 + L^2*c*t - L*c^2*t - L^2*l*t + L^2*l*x - 2*L*c*l*x - L*c*t*x + 2*c^2*t*x - 2*L*l*v*y + 2*c*t*v*y)*(L*c*l^2 - 2*L*c*l*t + L*l^2*t - c*l^2*t + c^2*t^2 - L*l^2 + L*c*t - c^2*t + 2*c*l*t - c*t^2)*(n + 1)/((L*c*l - L*c*t + L*l*t - c*l*t - L*l + c*t)*(L - c)*(c - l)*(l - t)*v)


**Some remarks:** The above expressions no longer involve the variable `r`. Moreover, the variable `v` only appears as a simple factor in the denominator, except for `T01` where it appears multiplying every instance of the variable `y`.

### Transform the matrix ICB to $(l,c)$ variables

In [110]:
ICB = inverse_change_base.subs(to_cl_vars)

In [111]:
b01 = simplify_vr(ICB[1][0])
b02 = simplify_vr(ICB[2][0])
b11 = simplify_vr(ICB[1][1])
b12 = simplify_vr(ICB[2][1])

In [112]:
ICM_final = matrix([(1, 0, 0), (b01, b11, 0), (b02, b12, 1)])
print ICM_final

[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [113]:
DM(ICM_final[1][0])

<IPython.core.display.Math object>

In [114]:
DM(ICM_final[2][0].numerator())
DM(ICM_final[2][0].denominator())

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [115]:
DM(ICM_final[1][1])

<IPython.core.display.Math object>

In [116]:
DM(ICM_final[2][1])

<IPython.core.display.Math object>

# The map into the moduli space of parabolic bundles $\mathbb{P}^1_z\times\mathbb{P}^1_w$.

Suppose we start with a parabolic bundle on $\mathbb{P}^1$ having parabolic structure $(0,1,c,\infty,l)$ over the points $0,1,\lambda,\infty,t$ respectively. After pulling back to the elliptic curve and mapping this parabolic bundle to its moduli space, we recover a point $(z,w) \in \mathbb{P}^1_z \times \mathbb{P}^1_w$. This point was explicitly computed by Nestor, and the values for $z,w$ are the following:

In [117]:
var('w')
dict['w'] = w

In [118]:
# Nestor's formula CORRECTED:
zw_vals = [
    z == L*(c-1)/(c-L),
    w == (L*l*(L*(l-1) + t*(1-c) + c - l))/(L*(t*(l-c) + l*(c-1)) + c*t*(1-l))
]

Frome the equations in the above section we can recover the value of (z,w) in terms of u,p,q.

In [119]:
var('mu1 mu2')
to_mu = [l==t*(mu1-1)/(mu1-t), c==L*(mu2-1)/(mu2-L)]

In [120]:
print w.subs(zw_vals).subs(to_mu).simplify_rational().factor()

(L*mu1 - mu2*t)*(mu1 - 1)/((mu1 - mu2)*(mu1 - t))


In [121]:
w_bis = w.subs(zw_vals).subs(to_mu).subs([mu1==W, mu2==p]).simplify_rational().factor()
print w_bis

(L*p*r*u - p^2*r*u - L*q*t*u + q*t^2*u - L*p*r + p^2*r - L*q*t + q*t^2 + L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)*(p*r*u - q*t*u - p*r - q*t)/((p - t)^2*q*r*(u + 1)*(u - 1))


### Simplify the expression for w

I don't like having `q` both in numerator and denominator, so let's multiply the above expression by `q/q` and replace `q^2` by `p*(p-1)*(p-L)`.  
Same goes for `r`.

In [122]:
new_w = simplify_qr(w_bis*r^2*q^2)/q2/r2

In [123]:
DM(new_w)

<IPython.core.display.Math object>

The new expression is still invariant under u -> 1/u, q -> -q

In [124]:
(new_w - new_w.subs([u==1/u, q==-q])).simplify_rational()

0

**Note:** The above expression is finally invariant under the substitution u -> -u !

In [125]:
(new_w - new_w.subs(u==-u)).simplify_rational()

0

Another symmetry: (from these three we can build up more)

In [126]:
(new_w - new_w.subs([u==1/u, r==-r])).simplify_rational()

0

**Question:** What happens when `u = 0`?

In [127]:
simple_l.subs(u==0).simplify_rational().factor()

(L*p*t - p^2*t - q*r - L*t + p*t)/((L - p)*(p - t))

In [128]:
W.subs(u==0).simplify_rational().factor()

(L*p*r - p^2*r + L*q*t - q*t^2)/(L*q + L*r - p*r - q*t)

In [129]:
new_w.subs(u==0).simplify_rational().factor()

-(2*L*p*t - p^2*t - p*t^2 - L*p - 2*q*r - L*t + 2*p*t)/(p - t)^2

**Remark:** B and C anti-commute. Conjugacy by a scalar matrix is the identity. Therefore, the actions "conjugate by B" and "conjugate by C" generate a group isomorphic to $\mathbb{Z}_2\times\mathbb{Z}_2$.

In [130]:
B*C*~B*~C

[-1  0]
[ 0 -1]

## Write the involution $u\mapsto -u$ in terms of the variables $\mu_i$

In [131]:
dict['mu1'] = mu1
dict['mu2'] = mu2

In [132]:
Sols_mu = solve([W - mu1, p - mu2], [u, p])[0]

In [133]:
foo = (W.subs(u==-u)).subs(Sols_mu).simplify_rational().factor()

In [134]:
bar = simplify_qr(foo).subs(p==mu2).simplify_rational().factor()

In [135]:
DM(bar)

<IPython.core.display.Math object>

The above expression has a factor of mu2 and t. It seems different from $W(-u)$, but note that if we make p=q=0 or t=r=0 then W vanishes identically.

We can verify that the above is in fact an involution:

In [136]:
bar.subs(mu1==bar).simplify_rational()

mu1